In [8]:
import os
import glob
import numpy as np
import cv2
import rasterio
from rasterio import features
import geopandas as gpd
import math
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# ==========================================
# CONFIGURATION: LAS VEGAS (SpaceNet AOI 2)
# ==========================================
CONFIG = {
    "SUN_AZIMUTH": 170.5,       # Direction of shadow (Degrees)
    "SUN_ELEVATION": 43.2,      # Sun height (Degrees)
    "PIXEL_RES": 0.3,           # Meters per pixel (WorldView-3)
    "FLOOR_HEIGHT": 3.2,        # Avg height of a floor (meters)
    "MIN_SHADOW_PX": 5,         # Noise filter (ignore tiny blobs)
    "SEARCH_DIST_M": 25.0,      # Search zone length (meters)
    "IMG_DIR": "PS-RGB",        # Input Folder (Images)
    "LABEL_DIR": "geojson_buildings", # Input Folder (Footprints)
    "OUTPUT_DIR": "evidence_reports"  # Output Folder
}

# ==========================================
# MODULE 1: ROBUST NORMALIZATION
# ==========================================
def normalize_to_8bit_robust(img_16bit):
    """
    Converts 16-bit satellite data to 8-bit for OpenCV.
    Uses Percentile Clipping (1%-99%) to fix the 'Black Image' issue.
    """
    img = np.ascontiguousarray(img_16bit)
    low_val = np.percentile(img, 1)
    high_val = np.percentile(img, 99)
    img_clipped = np.clip(img, low_val, high_val)
    img_8bit = cv2.normalize(img_clipped, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return img_8bit

# ==========================================
# MODULE 2: SHADOW DETECTION (TSAI)
# ==========================================
def tsai_shadow_detection(img_rgb):
    """
    Implementation of Tsai (2006) Spectral Ratio Algorithm.
    Includes Contrast Stretching to work in low-contrast environments (Deserts).
    """
    # 1. Convert to HLS (Hue, Lightness, Saturation)
    hls = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HLS)
    H, L, S = cv2.split(hls)
    
    # 2. Normalize components
    H_norm = H.astype(float) / 179.0
    I_norm = L.astype(float) / 255.0
    
    # 3. Calculate Spectral Ratio
    # Formula: (Hue + 1) / (Intensity + 1)
    ratio = (H_norm + 1) / (I_norm + 0.05)
    
    # 4. Contrast Stretching (The "Boost")
    # Forces the ratio map to use the full 0-255 range
    ratio_norm = cv2.normalize(ratio, None, 0, 255, cv2.NORM_MINMAX)
    ratio_norm = np.uint8(ratio_norm)
    
    # 5. Otsu's Thresholding
    _, shadow_mask = cv2.threshold(ratio_norm, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 6. Cleanup (Morphological Opening)
    kernel = np.ones((3,3), np.uint8)
    clean_mask = cv2.morphologyEx(shadow_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    
    return clean_mask

# ==========================================
# MODULE 3: EVIDENCE (PARALLEL LINE SCAN)
# ==========================================
def parallel_line_scan(shadow_mask, sun_azimuth):
    """
    The 'Parallel Scan' method from Gavankar (Paper 8).
    Rotates the shadow and scans for the maximum continuous length.
    """
    rows, cols = shadow_mask.shape
    
    # Rotate shadow to align with X-axis (remove Sun Angle)
    shadow_dir = (sun_azimuth + 180) % 360
    center = (cols // 2, rows // 2)
    M = cv2.getRotationMatrix2D(center, -shadow_dir, 1.0)
    rotated = cv2.warpAffine(shadow_mask, M, (cols, rows))
    
    max_len_px = 0
    
    # Scan every 2nd row
    for y in range(0, rows, 2):
        if np.sum(rotated[y, :]) == 0: continue
        
        contours, _ = cv2.findContours(rotated[y:y+1, :], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            length = cv2.arcLength(cnt, True) / 2
            if length > max_len_px:
                max_len_px = length
                
    return max_len_px

# ==========================================
# MODULE 4: MAIN PIPELINE
# ==========================================
def process_tile(tif_path, geojson_path, save_evidence=True):
    base_id = os.path.basename(tif_path).split('_')[-1].replace('.tif', '')
    print(f"Processing ID: {base_id}...")
    
    # 1. Load Data
    with rasterio.open(tif_path) as src:
        img_raw = src.read([1, 2, 3])
        img_raw = np.moveaxis(img_raw, 0, -1)
        img_8bit = normalize_to_8bit_robust(img_raw)
        transform = src.transform
        crs = src.crs
    
    try:
        gdf = gpd.read_file(geojson_path)
        if gdf.crs != crs: gdf = gdf.to_crs(crs)
    except:
        print("  ❌ GeoJSON Error")
        return

    # 2. Run Shadow Detection (Tsai)
    global_shadow_mask = tsai_shadow_detection(img_8bit)
    
    # 3. Create Visualization Canvas
    if save_evidence:
        vis_img = img_8bit.copy()
        vis_shadows = np.zeros_like(global_shadow_mask)

    results = []
    
    # 4. Iterate Buildings
    for idx, row in gdf.iterrows():
        # A. Rasterize Building Footprint
        geom = [row.geometry]
        bld_mask = features.rasterize(geom, out_shape=img_8bit.shape[:2], transform=transform, fill=0, default_value=1, dtype=np.uint8)
        
        # B. Define Search Zone (Directional)
        shadow_azimuth_rad = math.radians((CONFIG["SUN_AZIMUTH"] + 180) % 360)
        dist_px = CONFIG["SEARCH_DIST_M"] / CONFIG["PIXEL_RES"]
        dx = int(dist_px * math.sin(shadow_azimuth_rad))
        dy = int(dist_px * math.cos(shadow_azimuth_rad)) * -1
        
        M_shift = np.float32([[1, 0, dx], [0, 1, dy]])
        shifted_mask = cv2.warpAffine(bld_mask, M_shift, (img_8bit.shape[1], img_8bit.shape[0]))
        search_zone = cv2.subtract(shifted_mask, bld_mask)
        
        # C. Filter Shadows
        valid_shadow = cv2.bitwise_and(global_shadow_mask, global_shadow_mask, mask=search_zone)
        valid_shadow = cv2.subtract(valid_shadow, bld_mask * 255) # Clean Roofs
        
        # D. Measure (Parallel Scan)
        px_len = parallel_line_scan(valid_shadow, CONFIG["SUN_AZIMUTH"])
        
        if px_len > CONFIG["MIN_SHADOW_PX"]:
            # E. Physics
            shadow_m = px_len * CONFIG["PIXEL_RES"]
            height_m = shadow_m * math.tan(math.radians(CONFIG["SUN_ELEVATION"]))
            floors = round(height_m / CONFIG["FLOOR_HEIGHT"])
            
            results.append({"id": idx, "height": height_m, "floors": floors})
            
            # Add to evidence visualization
            if save_evidence:
                vis_shadows = cv2.bitwise_or(vis_shadows, valid_shadow)

    # 5. Report & Save
    if results:
        print(f"  ✅ Found {len(results)} Buildings with Shadows.")
        for r in sorted(results, key=lambda x: x['floors'], reverse=True)[:3]:
            print(f"     - Bld {r['id']}: {r['floors']} Floors ({r['height']:.2f}m)")
        
        if save_evidence:
            save_evidence_image(base_id, img_8bit, gdf, vis_shadows, transform)
    else:
        print("  ⚠️ No significant shadows found.")

def save_evidence_image(base_id, img, gdf, shadow_mask, transform):
    """Generates the Red/Blue overlay image for the report."""
    os.makedirs(CONFIG["OUTPUT_DIR"], exist_ok=True)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    
    # Draw Buildings (Blue)
    bld_mask = features.rasterize([(g, 1) for g in gdf.geometry], out_shape=img.shape[:2], transform=transform)
    plt.imshow(bld_mask, cmap=ListedColormap(['none', 'cyan']), alpha=0.4, interpolation='none')
    
    # Draw Shadows (Red)
    plt.imshow(shadow_mask, cmap=ListedColormap(['none', 'red']), alpha=0.6, interpolation='none')
    
    plt.title(f"ShadowAudit Evidence: {base_id}\nTsai Detection + Parallel Scan", fontsize=14)
    plt.axis('off')
    
    save_path = os.path.join(CONFIG["OUTPUT_DIR"], f"evidence_{base_id}.png")
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"  📸 Evidence saved to: {save_path}")

# ==========================================
# EXECUTION
# ==========================================
if __name__ == "__main__":
    # Auto-find files
    tif_files = sorted(glob.glob(os.path.join(CONFIG["IMG_DIR"], "*.tif")))
    geojson_files = sorted(glob.glob(os.path.join(CONFIG["LABEL_DIR"], "*.geojson")))
    
    print(f"🚀 Starting ShadowAudit POC on {len(tif_files)} files...")
    
    for t_path in tif_files[:]: # Run first 3 as demo
        base_id = os.path.basename(t_path).split('_')[-1].replace('.tif', '')
        g_path = next((g for g in geojson_files if base_id in g), None)
        
        if g_path:
            process_tile(t_path, g_path)

🚀 Starting ShadowAudit POC on 10 files...
Processing ID: img1...
  ⚠️ No significant shadows found.
Processing ID: img10...
  ✅ Found 8 Buildings with Shadows.
     - Bld 5: 1 Floors (3.66m)
     - Bld 7: 1 Floors (1.69m)
     - Bld 19: 1 Floors (3.38m)
  📸 Evidence saved to: evidence_reports\evidence_img10.png
Processing ID: img1002...
  ⚠️ No significant shadows found.
Processing ID: img1003...
  ✅ Found 6 Buildings with Shadows.
     - Bld 1: 5 Floors (17.18m)
     - Bld 3: 4 Floors (14.37m)
     - Bld 5: 4 Floors (13.24m)
  📸 Evidence saved to: evidence_reports\evidence_img1003.png
Processing ID: img1004...
  ✅ Found 12 Buildings with Shadows.
     - Bld 11: 5 Floors (16.62m)
     - Bld 3: 4 Floors (12.40m)
     - Bld 6: 4 Floors (13.52m)
  📸 Evidence saved to: evidence_reports\evidence_img1004.png
Processing ID: img1006...
  ✅ Found 12 Buildings with Shadows.
     - Bld 7: 7 Floors (23.95m)
     - Bld 15: 5 Floors (14.93m)
     - Bld 16: 5 Floors (16.90m)
  📸 Evidence saved to: ev